![Two data scientists working on a dashboard.](./Images/hr-image-small.png)

A common problem when creating models to generate business value from data is that the datasets can be so large that it can take days for the model to generate predictions. Ensuring that your dataset is stored as efficiently as possible is crucial for allowing these models to run on a more reasonable timescale without having to reduce the size of the dataset.

You've been hired by a major online data science training provider called *Training Data Ltd.* to clean up one of their largest customer datasets. This dataset will eventually be used to predict whether their students are looking for a new job or not, information that they will then use to direct them to prospective recruiters.

You've been given access to `customer_train.csv`, which is a subset of their entire customer dataset, so you can create a proof-of-concept of a much more efficient storage solution. The dataset contains anonymized student information, and whether they were looking for a new job or not during training:

| Column                   | Description                                                                      |
|------------------------- |--------------------------------------------------------------------------------- |
| `student_id`             | A unique ID for each student.                                                    |
| `city`                   | A code for the city the student lives in.                                        |
| `city_development_index` | A scaled development index for the city.                                         |
| `gender`                 | The student's gender.                                                            |
| `relevant_experience`    | An indicator of the student's work relevant experience.                          |
| `enrolled_university`    | The type of university course enrolled in (if any).                              |
| `education_level`        | The student's education level.                                                   |
| `major_discipline`       | The educational discipline of the student.                                       |
| `experience`             | The student's total work experience (in years).                                  |
| `company_size`           | The number of employees at the student's current employer.                       |
| `company_type`           | The type of company employing the student.                                       |
| `last_new_job`           | The number of years between the student's current and previous jobs.             |
| `training_hours`         | The number of hours of training completed.                                       |
| `job_change`             | An indicator of whether the student is looking for a new job (`1`) or not (`0`). |

In [125]:
# Import necessary libraries
import pandas as pd

# Load the dataset
ds_jobs = pd.read_csv("./Dataset/customer_train.csv")

# View the dataset
ds_jobs.head()

,student_id,city,city_development_index,gender,relevant_experience,enrolled_university,education_level,major_discipline,experience,company_size,company_type,last_new_job,training_hours,job_change
0,8949,city_103,0.920,Male,Has relevant experience,no_enrollment,Graduate,STEM,>20,NaN,NaN,1,36,1.0
1,29725,city_40,0.776,Male,No relevant experience,no_enrollment,Graduate,STEM,15,50-99,Pvt Ltd,>4,47,0.0
2,11561,city_21,0.624,NaN,No relevant experience,Full time course,Graduate,STEM,5,NaN,NaN,never,83,0.0
3,33241,city_115,0.789,NaN,No relevant experience,NaN,Graduate,Business Degree,<1,NaN,Pvt Ltd,never,52,1.0
4,666,city_162,0.767,Male,Has relevant experience,no_enrollment,Masters,STEM,>20,50-99,Funded Startup,4,8,0.0


In [126]:
# Create a copy of ds_jobs for transforming.dropna()
ds_jobs_transformed = ds_jobs.copy()

# Transform the dataset
ds_jobs_transformed['student_id'] = ds_jobs_transformed['student_id'].astype('uint16')
ds_jobs_transformed['training_hours'] = ds_jobs_transformed['training_hours'].astype('uint16')

ds_jobs_transformed['city_development_index'] = ds_jobs_transformed['city_development_index'].astype('float16')

ds_jobs_transformed['city'] = ds_jobs_transformed['city'].str.split('_').str[1].astype('uint8')

ds_jobs_transformed['gender'] = ds_jobs_transformed['gender'].astype('category')
ds_jobs_transformed['relevant_experience'] = ds_jobs_transformed['relevant_experience'].astype('category')
ds_jobs_transformed['major_discipline'] = ds_jobs_transformed['major_discipline'].astype('category')
ds_jobs_transformed['company_type'] = ds_jobs_transformed['company_type'].astype('category')

ds_jobs_transformed['job_change'] = ds_jobs_transformed['job_change'].astype('boolean')

relevant_experience_dict = {
    'No relevant experience': False,
    'Has relevant experience': True
}
ds_jobs_transformed['relevant_experience'] = ds_jobs_transformed['relevant_experience'].map(relevant_experience_dict).astype('boolean')


# Define the desired order
education_level_order = ['Unknown','Primary School', 'High School', 'Graduate', 'Masters', 'Phd']
ds_jobs_transformed['education_level'] = ds_jobs_transformed['education_level'].fillna('Unknown')
# Set as ordered categorical
ds_jobs_transformed['education_level'] = pd.Categorical(
    ds_jobs_transformed['education_level'],
    categories=education_level_order,
    ordered=True
)

# Define the desired order
enrolled_university_order = ['Unknown','no_enrollment', 'Part time course', 'Full time course']
ds_jobs_transformed['enrolled_university'] = ds_jobs_transformed['enrolled_university'].fillna('Unknown')
# Set as ordered categorical
ds_jobs_transformed['enrolled_university'] = pd.Categorical(
    ds_jobs_transformed['enrolled_university'],
    categories=enrolled_university_order,
    ordered=True
)

# Define the desired order
experience_order = ['Unknown','<1', '1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12', '13', '14', '15', '16', '17', '18', '19', '20', '>20']
ds_jobs_transformed['experience'] = ds_jobs_transformed['experience'].fillna('Unknown')
# Set as ordered categorical
ds_jobs_transformed['experience'] = pd.Categorical(
    ds_jobs_transformed['experience'],
    categories=experience_order,
    ordered=True
)

# Define the desired order
company_size_order = ['Unknown','<10', '10/49', '50-99', '100-500', '500-999', '1000-4999', '5000-9999', '10000+']
ds_jobs_transformed['experience'] = ds_jobs_transformed['experience'].fillna('Unknown')
# Set as ordered categorical
ds_jobs_transformed['company_size'] = pd.Categorical(
    ds_jobs_transformed['company_size'],
    categories=company_size_order,
    ordered=True
)

# Define the desired order
last_new_job_order = ['Unknown','1', '2', '3', '4', '>5']
ds_jobs_transformed['last_new_job'] = ds_jobs_transformed['last_new_job'].fillna('Unknown')
# Set as ordered categorical
ds_jobs_transformed['last_new_job'] = pd.Categorical(
    ds_jobs_transformed['last_new_job'],
    categories=last_new_job_order,
    ordered=True
)

In [127]:
ds_jobs_transformed['education_level'].value_counts()

education_level
Graduate          11598
Masters            4361
High School        2017
Unknown             460
Phd                 414
Primary School      308
Name: count, dtype: int64

In [128]:
ds_jobs_transformed = ds_jobs_transformed[
    (ds_jobs_transformed['experience'] >= '10') & 
    (ds_jobs_transformed['experience'] != 'Unknown') &
    (ds_jobs_transformed['company_size'] >= '1000-4999') & 
    (ds_jobs_transformed['company_size'] != 'Unknown')
]

ds_jobs_transformed.info()
ds_jobs.info()

<class 'pandas.core.frame.DataFrame'>
Index: 2201 entries, 9 to 19143
Data columns (total 14 columns):
 #   Column                  Non-Null Count  Dtype   
---  ------                  --------------  -----   
 0   student_id              2201 non-null   uint16  
 1   city                    2201 non-null   uint8   
 2   city_development_index  2201 non-null   float16 
 3   gender                  1821 non-null   category
 4   relevant_experience     2201 non-null   boolean 
 5   enrolled_university     2201 non-null   category
 6   education_level         2201 non-null   category
 7   major_discipline        2097 non-null   category
 8   experience              2201 non-null   category
 9   company_size            2201 non-null   category
 10  company_type            2144 non-null   category
 11  last_new_job            1299 non-null   category
 12  training_hours          2201 non-null   uint16  
 13  job_change              2201 non-null   boolean 
dtypes: boolean(2), category(8), 